## Load

In [1]:
from optimize import *
from full_model import *

c:\anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



Starting MATLAB engine...
-------------------------
Elapsed: 35.72 sec


In [2]:
data_loaded = loadmat("./saved/rescaled/Coil20")
images_norm = data_loaded['X']
images_norm_val = data_loaded['X_val']
labels = data_loaded['Y'].reshape(-1)
labels_val = data_loaded['Y_val'].reshape(-1)

In [3]:
load_path = "./saved/models/coil20/256.64_10000.10.3_0.2098"
hidden_dims = [256,64,256]

In [4]:
result1 = load("./optims/train/1_SSC/256.64_10000.10.3.opt")
result2 = load("./optims/train/2_SSC+AE/256.64_10000.10.3.opt")
result3 = load("./optims/train/3_Global/256.64_10000.10.3.opt")
result4 = load("./optims/train/4_Glob+Symm/256.64_10000.10.3.opt")
result5 = load("./optims/train/5_Conjoin/256.64_10000.10.3.opt")
result6 = load("./optims/train/6_Conj+Init/256.64_10000.10.3.opt")

In [5]:
results = [result1, result2, result3, result4, result5, result6]

## Re-eval

In [6]:
optimums = [res_optimum(result)[0] for result in results]

In [7]:
models = [run_ssc, run_ae, run_model, run_model, run_model, run_model]

In [8]:
spaces = [['alpha'],
          ['lr', 'batch_num', 'lambda2', 'alpha2'],
          ['lr', 'batch_num', 'alpha1', 'lambda1', 'lambda2', 'alpha2'],
          ['lr', 'batch_num', 'alpha1', 'lambda1', 'lambda2', 'alpha2'],
          ['lr', 'batch_num', 'lambda1', 'lambda2', 'lambda3'],
          ['lr', 'batch_num', 'alpha1', 'lambda1', 'lambda2', 'lambda3']]

In [9]:
params = [{'images_norm':images_norm, 'labels':labels},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims, 'trainC':False, 'symmC':False},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims, 'trainC':False, 'symmC':True},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims, 'trainC':True, 'giveC':False},
          {'images_norm':images_norm, 'images_norm_val':images_norm_val, 'labels':labels, \
           'load_path':load_path, 'hidden_dims':hidden_dims, 'trainC':True, 'giveC':False}]

In [13]:
from copy import copy

def run(idx, hypers=None, seed=0, verbose=True):
    if(hypers is None):
        hypers = optimums[idx]
    par = copy(params[idx])
    par.update(zip(spaces[idx], hypers))
    return models[idx](seed=seed, verbose=verbose, **par)

In [38]:
def run_mult(idx, hypers=None, seeds=range(100, 105), verbose=False):
    if(hypers is None):
        hypers = optimums[idx]
    
    log_sum = 0
    values = []
    
    for seed in seeds:
        value = run(idx, hypers, seed, verbose)
        log_sum += np.log(value)
        values.append(value)
    mean = np.exp(log_sum / len(seeds))
    
    diff_sum = 0
    for value in values:
        diff_sum += np.log(value / mean) ** 2
    std = np.exp(np.sqrt(diff_sum / len(seeds)))
    
    return mean, std

In [37]:
for idx in range(6):
    print(idx, run_mult(idx, seeds=range(100, 130)))

0 (array([0.15255458, 0.05621323, 0.19031127]), array([1.02554728, 1.05345709, 1.05838865]))
1 (array([0.13918033, 0.04691573, 0.17081291]), array([1.04099483, 1.01683183, 1.0124997 ]))
2 (array([0.19403063, 0.07919506, 0.2416942 ]), array([1.18145403, 1.12877655, 1.21602755]))
3 (array([0.21314304, 0.06540353, 0.22601901]), array([1.06851282, 1.1038804 , 1.12400808]))


c:\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


4 (array([0.34570456, 0.24199661, 0.41580567]), array([1.06879385, 1.08034589, 1.0700674 ]))
5 (array([0.33234488, 0.24583892, 0.41173368]), array([1.09216602, 1.08469467, 1.08538846]))


## Sensitivity analysis

In [21]:
from copy import copy

mults = [10**-1, 10**-0.5, 10**0.5, 10**1]
for idx in range(len(results)):
    # for each model
    xs = []
    ys = []
    hypers = copy(optimums[idx])
    for i in range(len(spaces[idx])):
        # for each hyperparameter
        x = []
        y = []
        hyper = hypers[i]
        for mult in mults:
            # for each multiplied value of said hyperparameter
            new_hyper = hyper * mult
            if(spaces[idx][i] == 'batch_num'):
                new_hyper = int(new_hyper)
                if(new_hyper <= 0):
                    new_hyper = 1
            hypers[i] = new_hyper
            x.append(new_hyper)
            y.append(run_mult(idx, hypers, seeds=range(100, 105)))
        hypers[i] = hyper
        xs.append(x)
        ys.append(y)
    print(idx, xs, ys)

0 [[0.7354101963496097, 2.325571234976412, 23.25571234976412, 73.54101963496096]] [[array([0.94444444, 0.91872887, 0.9996679 ]), array([0.82690651, 0.63888266, 0.97736735]), array([0.24322518, 0.12210265, 0.31450403]), array([0.43633952, 0.2282624 , 0.642591  ])]]
1 [[4.004277562598502e-06, 1.2662637481318733e-05, 0.00012662637481318734, 0.00040042775625985023], [2, 6, 63, 200], [0.002694794548503328, 0.008521688599475608, 0.08521688599475609, 0.26947945485033276], [0.29412978394240047, 0.9301200449512051, 9.301200449512052, 29.412978394240046]] [[array([0.15034682, 0.05050148, 0.17719993]), array([0.1484375 , 0.04830346, 0.17462973]), array([0.15949283, 0.05480618, 0.19216263]), array([0.13847535, 0.06508809, 0.25139915])], [array([0.14009817, 0.05061405, 0.17506718]), array([0.14079282, 0.05024343, 0.17509831]), array([0.13721832, 0.0467222 , 0.16984892]), array([0.14096834, 0.05053155, 0.1796922 ])], [array([0.15585382, 0.04866179, 0.17920832]), array([0.17001367, 0.05011462, 0.1851

c:\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


4 [[0.0002636778074047182, 0.0008338224398381208, 0.008338224398381208, 0.026367780740471816], [1, 1, 12, 40], [0.12197508538591706, 0.3857190876130161, 3.857190876130161, 12.197508538591705], [9.037961288138778e-05, 0.00028580543074947887, 0.002858054307494789, 0.009037961288138777], [75.079256151631, 237.42145447036208, 2374.214544703621, 7507.9256151631]] [[array([0.37574675, 0.27279462, 0.46036837]), array([0.35703653, 0.25655805, 0.43980386]), array([0.38150336, 0.27798127, 0.46508142]), array([0.60192431, 0.60638413, 0.76474539])], [array([0.38010703, 0.2647863 , 0.45985658]), array([0.38010703, 0.2647863 , 0.45985658]), array([0.37531015, 0.28124531, 0.46199893]), array([0.40806951, 0.3248356 , 0.52060303])], [array([0.35978478, 0.26525376, 0.44929495]), array([0.3556135 , 0.24597259, 0.4287906 ]), array([0.3605308 , 0.25532803, 0.44326583]), array([0.34466677, 0.25001647, 0.41451403])], [array([0.35203035, 0.23653527, 0.41318626]), array([0.34500979, 0.23649211, 0.40920635]), a